In [1]:
import pandas as pd
from music21 import converter, instrument, note, chord
from sklearn.preprocessing import LabelEncoder
import itertools
import warnings
warnings.filterwarnings("ignore")

In [2]:
path = 'data\maestro-v3.0.0.csv'
df = pd.read_csv(path)

In [5]:
#df = df.sort_values(by='duration').iloc[:50]
#df = df[df['canonical_composer'].isin(df['canonical_composer'].unique()[:10])]
#compositeur avec le plus de track 
comp = df.groupby('canonical_composer')['midi_filename'].count().reset_index().sort_values(by='midi_filename')['canonical_composer'].to_list()[-10:-9]
df = df[df['canonical_composer'].isin(comp)]

In [6]:
def extraction_notes(file):
    midi = converter.parse(file)
    L_notes = []
    for e in midi.recurse().notes:
        if e.isNote:
            L_notes.append(e.pitch.midi)
        elif e.isChord:
            L_notes.append(e.pitches[0].midi)
    if len(L_notes)<8:
        return 'null'
    else:
        return L_notes 
    
def ex_notes_tab(df,col_out):
    df['midi_file_data'] = 'data/'+df['midi_filename']
    df[col_out] = df['midi_file_data'].apply(extraction_notes)
    return df[df[col_out]!='null']

In [7]:
def colonnes(df):
    df_expanded = df['X'].apply(pd.Series)
    df_expanded['y'] = df['y']
    return df_expanded

def sequence_list(L):
    sequence_length = 8

    input_sequences = []
    output_notes = []

    for i in range(len(L) - sequence_length):
        seq_in = L[i:i + sequence_length]  
        seq_out = L[i + sequence_length]  
        input_sequences.append(seq_in)
        output_notes.append(seq_out)

    return input_sequences,output_notes

def sequence(dff):
    X,y = [],[]
    for k in dff['notes'].to_list():
        xtemp,ytemp = sequence_list(k)
        X.append(xtemp)
        y.append(ytemp)
    X=list(itertools.chain(*X))
    y=list(itertools.chain(*y))

    return pd.DataFrame({"X":X,"y":y})

In [8]:
def preprocessing(df):

    dftrain = colonnes(sequence(ex_notes_tab(df[df['split']=='train'],'notes')))
    dftest = colonnes(sequence(ex_notes_tab(df[df['split']=='test'],'notes')))
    dfeval = colonnes(sequence(ex_notes_tab(df[df['split']=='validation'],'notes')))

    return dftrain,dftest,dfeval
    

In [9]:
dftrain,dftest,deval = preprocessing(df)

In [10]:
dftrain.to_csv('00_train_V4_8_temps_3.csv',index=False)
dftest.to_csv('00_test_V4_8_temps_3.csv',index=False)
deval.to_csv('00_validation_V4_8_temps_3.csv',index=False)